<a href="https://colab.research.google.com/github/CL500Coupe/data_analytics_assi1/blob/main/assi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import sys
import os
import pandas as pd
import seaborn as sns
import pickle
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [172]:
url = "https://raw.githubusercontent.com/CL500Coupe/data_analytics_assi1/main/data/final_data.csv"
data = pd.read_csv(url)

In [173]:
# Convert the 'collision_date' column to datetime format
data['collision_date'] = pd.to_datetime(data['collision_date'])

# Set the 'collision_date' column as the index
data = data.set_index('collision_date')

In [174]:
# List of columns with NA values to fill with mean values of the last 7 available days
columns_with_na_7 = [
    "visib_7day_ma",
    "wdsp_7day_ma",
    "co2_7day_ma",
    "so2_7day_ma",
    "pressure_7day_ma",
    "temp_7day_ma",
    "wind_7day_ma",
]

# List of columns with NA values to fill with mean values of the last 30 available days
columns_with_na_30 = [
    "visib_30day_ma",
    "wdsp_30day_ma",
    "co2_30day_ma",
    "so2_30day_ma",
    "pressure_30day_ma",
    "temp_30day_ma",
    "wind_30day_ma",
]

# Fill NA values with the mean values of the first 7 available days
for col in columns_with_na_7:
    first_valid_index = data[col].first_valid_index()
    mean_value = data.loc[first_valid_index:first_valid_index + pd.Timedelta(days=6), col].mean()
    data[col] = data[col].fillna(mean_value, limit=6)

# Fill NA values with the mean values of the first 30 available days
for col in columns_with_na_30:
    first_valid_index = data[col].first_valid_index()
    mean_value = data.loc[first_valid_index:first_valid_index + pd.Timedelta(days=29), col].mean()
    data[col] = data[col].fillna(mean_value, limit=29)

# Display the updated DataFrame
print(data.head())



                day  year  mo  temp  dewp     slp  visib  wdsp  mxpsd  gust  \
collision_date                                                                
2012-07-01        7  2012   7  83.6  63.0  1008.9    9.7   4.1    9.9  18.1   
2012-07-02        1  2012   7  80.3  54.1  1011.6   10.0   3.8   15.0  18.1   
2012-07-03        2  2012   7  79.8  56.7  1012.8   10.0   2.9   12.0  18.1   
2012-07-04        3  2012   7  81.8  65.6  1009.1    9.2   3.6   11.1  18.1   
2012-07-05        4  2012   7  86.7  64.3  1007.4    9.4   3.8   15.0  18.1   

                ...  co2_30day_ma  so2_30day_ma  pressure_30day_ma  \
collision_date  ...                                                  
2012-07-01      ...     15.031652      1.210648         917.765355   
2012-07-02      ...     15.031652      1.210648         917.765355   
2012-07-03      ...     15.031652      1.210648         917.765355   
2012-07-04      ...     15.031652      1.210648         917.765355   
2012-07-05      ...     15

In [175]:
data.shape


(2969, 39)

In [176]:
data.head()

,day,year,mo,temp,dewp,slp,visib,wdsp,mxpsd,gust,...,co2_30day_ma,so2_30day_ma,pressure_30day_ma,temp_30day_ma,wind_30day_ma,temp_visib_interaction,wdsp_visib_interaction,prcp_temp_interaction,temp_season_interaction,wdsp_wind_interaction
collision_date,,,,,,,,,,,,,,,,,,,,,
2012-07-01,7,2012,7,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,810.92,39.77,0.000,167.2,553.145520
2012-07-02,1,2012,7,80.3,54.1,1011.6,10.0,3.8,15.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,803.00,38.00,0.000,160.6,545.418750
2012-07-03,2,2012,7,79.8,56.7,1012.8,10.0,2.9,12.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,798.00,29.00,0.000,159.6,362.080104
2012-07-04,3,2012,7,81.8,65.6,1009.1,9.2,3.6,11.1,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,752.56,33.12,4.908,163.6,453.314999
2012-07-05,4,2012,7,86.7,64.3,1007.4,9.4,3.8,15.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,814.98,35.72,8669.133,173.4,479.916250


In [177]:
data.dtypes

day                          int64
year                         int64
mo                           int64
temp                       float64
dewp                       float64
slp                        float64
visib                      float64
wdsp                       float64
mxpsd                      float64
gust                       float64
max                        float64
min                        float64
prcp                       float64
fog                          int64
NUM_COLLISIONS               int64
co2                        float64
wind                       float64
so2                        float64
pressure                   float64
season                       int64
visib_7day_ma              float64
wdsp_7day_ma               float64
co2_7day_ma                float64
so2_7day_ma                float64
pressure_7day_ma           float64
temp_7day_ma               float64
wind_7day_ma               float64
visib_30day_ma             float64
wdsp_30day_ma       

In [178]:
data.columns

Index(['day', 'year', 'mo', 'temp', 'dewp', 'slp', 'visib', 'wdsp', 'mxpsd',
       'gust', 'max', 'min', 'prcp', 'fog', 'NUM_COLLISIONS', 'co2', 'wind',
       'so2', 'pressure', 'season', 'visib_7day_ma', 'wdsp_7day_ma',
       'co2_7day_ma', 'so2_7day_ma', 'pressure_7day_ma', 'temp_7day_ma',
       'wind_7day_ma', 'visib_30day_ma', 'wdsp_30day_ma', 'co2_30day_ma',
       'so2_30day_ma', 'pressure_30day_ma', 'temp_30day_ma', 'wind_30day_ma',
       'temp_visib_interaction', 'wdsp_visib_interaction',
       'prcp_temp_interaction', 'temp_season_interaction',
       'wdsp_wind_interaction'],
      dtype='object')

In [179]:
# Check if there are any missing values in the entire DataFrame
has_missing_values = data.isna().any().any()

# Print the result
print("Are there any missing values in the DataFrame?", has_missing_values)

Are there any missing values in the DataFrame? False


In [180]:
# Select the features and target variable
X = data.drop("NUM_COLLISIONS", axis=1)  # Replace "num_collisions" with the actual target column name
y = data["NUM_COLLISIONS"]  # Replace "num_collisions" with the actual target column name

In [ ]:
# Create a linear regression model
model = LinearRegression()

In [ ]:
# Compute the cross-validated MSE and MAE scores
mse_scores = -1 * cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
mae_scores = -1 * cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')

In [ ]:
# Compute the mean squared error (MSE), mean absolute error (MAE), and their square roots (RMSE, RMAE)
mse = mse_scores.mean()
rmse = mse_scores.mean() ** 0.5
mae = mae_scores.mean()


In [ ]:
print("Mean Squared Error: {:.4f}".format(mse))
print("Root Mean Squared Error: {:.4f}".format(rmse))
print("Mean Absolute Error: {:.4f}".format(mae))

Mean Squared Error: 64576.1269
Root Mean Squared Error: 254.1183
Mean Absolute Error: 156.9246


In [161]:
params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'selection': ['cyclic', 'random'],
    'max_iter': [1000, 5000, 10000],
    'tol': [0.0001, 0.001, 0.01],
}

In [162]:
# Create the Lasso regression model
lasso = Lasso()

In [163]:
# Create the GridSearchCV object
grid_search = GridSearchCV(lasso, param_grid=params, cv=10)

In [164]:
# Fit the model using the GridSearchCV object
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.434e+06, tolerance: 3.517e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.413e+05, tolerance: 3.389e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.161e+03, tolerance: 3.500e+03

GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10],
                         'max_iter': [1000, 5000, 10000],
                         'selection': ['cyclic', 'random'],
                         'tol': [0.0001, 0.001, 0.01]})

In [165]:
# Print the best parameters found
print('Best Parameters:', grid_search.best_params_)

Best Parameters: {'alpha': 0.001, 'max_iter': 1000, 'selection': 'random', 'tol': 0.001}


In [166]:
# Make predictions using the best model
y_pred = grid_search.predict(X_test)


In [167]:
# Compute the mean squared error (MSE), mean absolute error (MAE), and their square roots (RMSE, RMAE)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)


In [168]:
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)

Mean Squared Error: 36669.85895982583
Root Mean Squared Error: 191.49375697349987
Mean Absolute Error: 93.69948218684006


In [189]:
# Split the data into training and testing sets
X_train_DNN, X_test_DNN, y_train_DNN, y_test_DNN = train_test_split(X, y, test_size=0.2, random_state=42)

In [190]:
# Define the model
model_DNN = Sequential([
    Dense(64, activation='relu', input_shape=[X_train_DNN.shape[1]]),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

In [191]:
# Compile the model
model_DNN.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [192]:
# Train the model
history = model_DNN.fit(X_train_DNN, y_train_DNN, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
60/60 [==============================] - 1s 5ms/step - loss: 62792.6445 - mae: 185.4000 - val_loss: 20392.2441 - val_mae: 109.2400
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 28259.7539 - mae: 128.9646 - val_loss: 20176.6113 - val_mae: 102.0436
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 24857.0820 - mae: 119.8150 - val_loss: 18890.1797 - val_mae: 102.3080
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 24579.5156 - mae: 118.7704 - val_loss: 19101.5195 - val_mae: 101.7523
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 22976.0039 - mae: 115.6413 - val_loss: 18659.5586 - val_mae: 100.9188
Epoch 6/50
60/60 [==============================] - 0s 6ms/step - loss: 23786.7598 - mae: 117.1157 - val_loss: 18856.8203 - val_mae: 102.5273
Epoch 7/50
60/60 [==============================] - 0s 8ms/step - loss: 23509.4766 - mae: 119.2188 - val_loss: 18263.3301 - val_mae: 98.6006
Epoch 8

In [196]:
# Use the trained DNN model to predict on the test set
y_pred = model_DNN.predict(X_test_DNN)

19/19 [==============================] - 0s 1ms/step


In [197]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

In [198]:
# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

In [199]:
# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

In [200]:
# Print the evaluation metrics
print("Mean Squared Error: {:.4f}".format(mse))
print("Root Mean Squared Error: {:.4f}".format(rmse))
print("Mean Absolute Error: {:.4f}".format(mae))

Mean Squared Error: 19514.6697
Root Mean Squared Error: 139.6949
Mean Absolute Error: 94.9869


In [ ]:
#TUNER INSTEAD OF GRID_SEARRCH